# **Machine Transliteration**

In [ ]:
from tensorflow.keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, \
  Bidirectional, RepeatVector, Concatenate, Activation, Dot, Lambda
import numpy as np
import keras.backend as K

batch_size = 64 # Batch size for Training
epochs = 100 # Number of epochs to train for
latent_dim = 256 #Latent dimentionality of the encoding space
num_sample = 15000 # Number of samples to train
#here change the path accordingly
data_path = 'crowd_transliterations.hi-en.txt'

In [ ]:
#preprocessing or cleaning of data

def preprocess_sentence_english(w):
  for ch in w:
    if((ch >= 'a' and ch <= 'z') or (ch >= 'A' and ch <= 'Z')):
      {}
    else:
      return 'false'
  return 'true'
def preprocess_sentence_hindi(w):
  for ch in w:
    if((ch == '-') or (ch == ',') or (ch >= 'a' and ch <= 'z') or (ch >= 'A' and ch <= 'Z') or (ch >= '0' and ch <= '9')):
      return 'false'
  return 'true'

In [ ]:
# vectorize the data 

input_texts = [] # english list
target_texts = [] # Hindi list
input_characters = set() #distinct input characters
target_characters = set() #distinct output characters
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
for line in lines[:len(lines)-1]:
  input_text,target_text = line.split('\t');
  res = preprocess_sentence_english(input_text);
  if(res == 'false'):
    continue
  res = preprocess_sentence_hindi(target_text);  
  if(res == 'false'):
    continue

  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text);
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char);
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char);

In [ ]:
input_characters = sorted(list(input_characters));
target_characters = sorted(list(target_characters));

#required for the creation of 3D arrays for LSTM
num_encoder_tokens = len(input_characters)
print(num_encoder_tokens);
num_decoder_tokens = len(target_characters)
print(num_decoder_tokens);
max_encoder_seq_length = max([len(word) for word in input_texts])
print(max_encoder_seq_length);
max_decoder_seq_length = max([len(word) for word in target_texts])
print(max_decoder_seq_length);

In [ ]:
# here we are providing the index to the input tokens and target tokens
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [ ]:
# initialize arrays with all zeroes to store one hot representation
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

In [ ]:
# create one hot representation
for i, (ip_text, tar_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(ip_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  
  for t, char in enumerate(tar_text):
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      decoder_target_data[i, t-1, target_token_index[char]] = 1.

In [ ]:
# encoder LSTM's
encoder_inputs = Input(shape=(max_encoder_seq_length, num_encoder_tokens))
encoder = Bidirectional(LSTM(latent_dim, return_sequences= True))
encoder_outputs = encoder(encoder_inputs)
print(encoder_outputs)
#encoder_states = [state_h, state_c] #because we did not required intermediate encoder outputs

In [ ]:
# decoder LSTM's
decoder_inputs = Input(shape=(max_decoder_seq_length, num_decoder_tokens))

In [ ]:
######### Attention #########
# Attention layers need to be global because
# they will be repeated Ty times at the decoder
attn_repeat_layer = RepeatVector(max_encoder_seq_length)
attn_concat_layer = Concatenate(axis=-1)
attn_dense1 = Dense(10, activation='tanh')
attn_dense2 = Dense(1, activation='softmax')
attn_dot = Dot(axes=1) # to perform the weighted sum of alpha[t] * h[t]

def one_step_attention(h, st_1):
  # h = h(1), ..., h(Tx), shape = (Tx, LATENT_DIM * 2)
  # st_1 = s(t-1), shape = (LATENT_DIM_DECODER,)
  # copy s(t-1) Tx times
  # now shape = (Tx, LATENT_DIM_DECODER)
  st_1 = attn_repeat_layer(st_1)
  # Concatenate all h(t)'s with s(t-1)
  # Now of shape (Tx, LATENT_DIM_DECODER + LATENT_DIM * 2)
  x = attn_concat_layer([h, st_1])
  # Neural net first layer
  x = attn_dense1(x)

  # Neural net second layer with special softmax over time
  alphas = attn_dense2(x)
  # "Dot" the alphas and the h's
  # Remember a.dot(b) = sum over a[t] * b[t]
  context = attn_dot([alphas, h])
  return context

In [ ]:
decoder_lstm = LSTM(latent_dim, return_state=True)
#decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
initial_s = Input(shape=(latent_dim,), name='s0')
initial_c = Input(shape=(latent_dim,), name='c0')
context_last_word_concat_layer = Concatenate(axis=2)

# s, c will be re-assigned in each iteration of the loop
s = initial_s
c = initial_c

In [ ]:
# collect outputs in a list at first
outputs = []
for t in range(max_decoder_seq_length): # Ty times
  # get the context using attention
  context = one_step_attention(encoder_outputs, s)
  # we need a different layer for each time step
  selector = Lambda(lambda x: x[:, t:t+1])
  xt = selector(decoder_inputs)
  
  # combine 
  decoder_lstm_input = context_last_word_concat_layer([context, xt])

  # pass the combined [context, last word] into the LSTM
  # along with [s, c]
  # get the new [s, c] and output
  o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[s, c])

  # final dense layer to get next word prediction
  decoder_outputs = decoder_dense(o)
  outputs.append(decoder_outputs)

#decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
def stack_and_transpose(x):
  # x is a list of length T, each element is a batch_size x output_vocab_size tensor
  x = K.stack(x) # is now T x batch_size x output_vocab_size tensor
  x = K.permute_dimensions(x, pattern=(1, 0, 2)) # is now batch_size x T x output_vocab_size
  return x

# make it a layer
stacker = Lambda(stack_and_transpose)
outputs = stacker(outputs)

In [ ]:
# model formation
model = Model(inputs = [encoder_inputs, decoder_inputs, initial_s, initial_c], outputs = outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

z = np.zeros((len(input_texts), latent_dim))
r = model.fit([encoder_input_data, decoder_input_data,z,z], decoder_target_data,
          batch_size = batch_size, 
          epochs=epochs, 
          validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

# plot some data
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

# accuracies
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()

In [ ]:
#creation of encoder model and decoder model 

encoder_model = Model(encoder_inputs, encoder_outputs)

encoder_outputs_as_input = Input(shape=(max_encoder_seq_length, latent_dim*2,))

decoder_inputs_single = Input(shape=(1,num_decoder_tokens))
print(decoder_inputs_single)

context = one_step_attention(encoder_outputs_as_input, initial_s)
# combine context with last word
decoder_lstm_input = context_last_word_concat_layer([context, decoder_inputs_single])

print(decoder_lstm_input)
# lstm and final dense
o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[initial_s, initial_c])
decoder_outputs = decoder_dense(o)

# create the model object
decoder_model = Model(
  inputs=[
    decoder_inputs_single,
    encoder_outputs_as_input,
    initial_s, 
    initial_c
  ],
  outputs=[decoder_outputs, s, c]
)

#reverse indexing in order to predicr output
reverse_input_char_index = dict((i, char) for char,i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char,i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    enc_out = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.
    s = np.zeros((1, latent_dim))
    c = np.zeros((1, latent_dim))

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        o, s, c = decoder_model.predict([target_seq, enc_out, s, c])
        # Sample a token
        sampled_token_index = np.argmax(o.flatten())
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

    return decoded_sentence

In [ ]:
# Testing
for seq_index in range(100):
  input_seq = encoder_input_data[seq_index: seq_index+1]
  decoded_word = decode_sequence(input_seq)
  print('Input word : ', input_texts[seq_index])
  print('Output word : ', decoded_word);

In [ ]:
# user input
user_In = input("enter input : ")
inp = np.zeros((1,max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
for t, char in enumerate(user_In):
  inp[0,t, input_token_index[char]] = 1.
decoded_word = decode_sequence(inp)
print('Input word : ', user_In)
print('Output word : ', decoded_word)